## 2c. Calculate closed- and open-system regassing paths

## Setting things up
First we need to import a few Python packages (including VolFe) and set the precision of the calculations. You need to install VolFe once on your machine, if you haven't yet, uncomment the line below (remove the #).

In [ ]:
# Install VolFe on your machine. Remove the # from line below to do this (don't remove the # from this line!).
# pip install VolFe

In [1]:
# import python packages
import pandas as pd
import gmpy2 as gp
import VolFe as vf

# set calculation precision
gp.get_context().precision=50

Instead of running a degassing calculation, where we decrease pressure simulating magma ascent, we can do a regassing calculation, where we increase pressure and resorb vapor into the melt.

For these types of calculations, we specify the melt composition at the start of the regassing path. 
We'll use Sari15-04-33 from Brounce et al. (2014) again assuming a temperature of 1200 'C.

For closed-system regassing calculations, we need an estimate of the bulk composition of the system - either intial CO2 or wt% gas present as described in Example 2b - at the start of regassing.
For this example we'll assume the initial melt contained 2 wt% CO2-eq.

Finally, we also need to state the final pressure of the calculation, which we specify in the inputs using "final_P" in bars.

So the input file looks like below:

In [2]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Sari15-04-33',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 47.89, # wt%
           'TiO2': 0.75, # wt%
           'Al2O3': 16.74, # wt%
           'FeOT': 9.43, # wt%
           'MnO': 0.18, # wt%
           'MgO': 5.92, # wt%
           'CaO': 11.58, # wt%
           'Na2O': 2.14, # wt%
           'K2O': 0.63, # wt%
           'P2O5': 0.17, # wt%
           'H2O': 4.17, # wt%
           'CO2ppm': 1487., # ppm
           'STppm': 1343.5, # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.195,
           'initial_CO2wtpc': 2., # initial CO2 content of the system in wt%
           'final_P':6000.} # bar

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

We also need to change some model options to say its a regassing - rather than degassing - calculation.
This is done by changing "gassing_direction" to "regas" (the default is "degas").
As before (Example 2b), to use the initial CO2 in the calculation, we have to change the "bulk_composition" option to "CO2" so it knows how to calculate the bulk composition, and change "starting_P" to "measured".
This gives the following models dataframe, assuming the defaults for everything else:

In [ ]:
# choose the options I want - everything else will use the default options
my_models = [['gassing_direction','regas'],['bulk_composition','CO2'],['starting_P','measured']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

VolFe will then calculate the bulk composition of the system by calculating the melt and vapor composition at Pvsat based on the input composition, and then add that vapor composition to the melt such that the bulk composition is 1 wt% CO2.
Then it will increase the pressure (default pressure step size of 10 bar so it might take a while to run) up to the final pressure.

In [ ]:
vf.calc_gassing(my_analysis, models=my_models)

We could have used wt% vapor present instead of initial CO2 content to calculate the bulk composition if preferred - see Example 2b for details.

### in progress ###

We can do the same assuming open-system degassing by changing the "gassing_direction" and "gassing_style"

In [3]:
# choose the options I want - everything else will use the default options
my_models = [['gassing_direction','regas'],['gassing_style','open'],['starting_P','measured']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

In [4]:
vf.calc_gassing(my_analysis, models=my_models)

Sari15-04-33
1200.0 3863.5831334191316 2024-04-06 22:59:34.335521


TypeError: can't multiply sequence by non-int of type 'float'

VolFe will then calculate the bulk composition of the system by calculating the melt and vapor composition at Pvsat based on the input composition, and then add 1 wt% of that vapor composition to the melt.

Again, the calculation below will run with the default pressure step size of 10 bar so it might take a while to run.

### in progress ###
Plotting...
